In [1]:
# Using RxNorm names and RxNorm synonyms to ChEMBL pref_name, compound_name, and synonyms
# Do this for concepts that had no DrugBank ID via RxNorm
# Also concepts tha thad a DrugBank ID but the InChI retrieved from Unichem did not exist in ChEMBL (because compounds doesn't exist in ChEMBL or because it has a different structure in ChEMBL)
# So need to select the compounds that are not in the current mapped_compounds db (from FAERS_compound_mapping.ipynb)

In [2]:
import sqlite3 as sqlite
import itertools
from sshtunnel import SSHTunnelForwarder
import pymysql
import sys
import pandas as pd

In [3]:
# Set parameteres
basedir = '/Users/ines/FAERS_y2'
# Location of the sqlite database used for the mapping process
mapping_process_db = basedir + '/data/interim/201903_drug_mapping_process.db'
# Location/name of the new database to be created for storing mapped compounds
mapped_compounds_db = basedir + '/data/interim/201903_mapped_compounds.db'

In [4]:
# Connect to sqlite db containing mapped IDs to find the concepts that have already been mapped

conn = sqlite.connect(mapped_compounds_db)
cur = conn.cursor()

In [5]:
aeolus_mapped = [i[0] for i in cur.execute('select distinct aeolus_concept from compound_structures').fetchall()]

In [6]:
len(aeolus_mapped)

1835

In [7]:
conn.close()

In [19]:
# Connect to sqlite db containing the rxnorm IDs

conn = sqlite.connect(mapping_process_db)
cur = conn.cursor()

In [9]:
# Total number or rows
cur.execute("select count(distinct rxnorm_concept) from drug_concepts").fetchall()

[(3503,)]

In [10]:
# Number of retired concepts
cur.execute("select count(distinct rxnorm_concept) from drug_concepts where rxnorm_comment='retired'").fetchall()

[(27,)]

In [11]:
# Number of biological
cur.execute("select count(distinct rxnorm_concept) from drug_concepts where drugbank_type='biotech' ").fetchall()

[(263,)]

In [24]:
# Select all the concepts currently not mapped or annotated as biological
# Including retired concepts, I guess could try map on name for those?
# SQL syntax is such that if 'not in' is used then entries with NULL are excluded, so 'or drugbank_comment is null' needs to be added to include those

unmapped_query = """select distinct 
aeolus_concept
, rxnorm_concept
, rxnorm_name
, drugbank_id
from drug_concepts 
where (drugbank_comment != 'biotech' or drugbank_comment is null)
and aeolus_concept not in ({})""".format(', '.join([str(i) for i in aeolus_mapped]))

unmapped = [(i[0], i[1], i[2], i[3]) for i in cur.execute(unmapped_query).fetchall()]

In [26]:
conn.close()

In [25]:
mapping_dict = {}
for item in unmapped:
    mapping_dict[item[0]] = {'aeolus_concept':item[0], 'rxnorm_concept': item[1], 'rxnorm_name': item[2], 'drugbank_id': item[3]}

In [27]:
len(mapping_dict)

1691

In [28]:
# Tiered system: 
# 1. RxNorm name against ChEMBL pref_name where max_phase = 4 (only approved compounds)
# 2. RxNorm name against compound_name,  max_phase = 4
# 3. RxNorm name against ChEMBL synonyms, max_phase = 4
# for 1, 2 and 3: if multiple compounds with that name, manually inspect if there was more than one match

# 4. RxNorm synonyms (excluding NDFRT, SNOMED_US) against ChEMBL pref_name, *not* max_phase = 4
# 5. RxNorm synonyms (excluding NDFRT, SNOMED_US) against ChEMBL synonyms, *not* max_phase = 4
# 6. RxNorm name and RxNorm synonyms against compound_name, max_phase 4

### Set up database connections - only works when on VPN to Chemistry department

In [49]:
with open('/Users/ines/ines_calculon_login.txt') as s: 
    calculon_pw = s.read().strip()
with open('/Users/ines/ines_sql_login.txt') as f: 
    sql_pw = f.read().strip()

In [86]:
server = SSHTunnelForwarder(
         ('calculon.ch.private.cam.ac.uk', 22), 
         ssh_password=calculon_pw,
         ssh_username="ias41",
         remote_bind_address=('127.0.0.1', 3306))

In [87]:
server.start()

In [85]:
server.close()

In [88]:
calculon_chembl = pymysql.connect(db='chembl_24', user='ias41', password=sql_pw, host = '127.0.0.1', unix_socket='/var/run/mysqld/mysqld.sock', port=server.local_bind_port) 

OperationalError: (2003, "Can't connect to MySQL server on '127.0.0.1' ([Errno 2] No such file or directory)")

In [21]:
def do_query(myquery):
    try:
        with calculon_chembl.cursor() as cursor:
            cursor.execute(myquery)
            result = cursor.fetchall()
            return(result)
    except:
        e = sys.exc_info()
        print(str(e))
        

### Start querying

In [22]:
# unmapped compounds
len(['"'+j+'"' for j in [mapping_dict[i]['rxnorm_name'].upper() for i in mapping_dict] if (not '(' in j and not ')' in j)])

1505

In [23]:
# Make string of RxNorm drug names currently unmapped
joined_names = ', '.join(['"'+j+'"' for j in [mapping_dict[i]['rxnorm_name'].upper() for i in mapping_dict] if (not '(' in j and not ')' in j)])

In [24]:
# Query 1 - RxNorm name to ChEMBL pref_name, max_phase = 4
pref_name_query = """select distinct
md.pref_name
, str.standard_inchi
, str.standard_inchi_key
, md.chembl_id
, md.max_phase
, str.molregno
, hier.parent_molregno
, str2.standard_inchi as parent_inchi
, str2.standard_inchi_key as parent_inchi_key
, md2.chembl_id as parent_chembl_id
, md2.max_phase as parent_max_phase
, md2.pref_name as parent_pref_name
from molecule_dictionary md 
left join compound_structures str on md.molregno = str.molregno
left join molecule_hierarchy hier on hier.molregno = str.molregno
left join compound_structures str2 on str2.molregno = hier.parent_molregno
left join molecule_dictionary md2 on md2.molregno = hier.parent_molregno
where md.pref_name in ({})
and md.max_phase = 4""".format(joined_names)

In [25]:
pref_name_result = do_query(pref_name_query)
print(len(pref_name_result))
pref_name_result[:5]

357


(('ABATACEPT',
  None,
  None,
  'CHEMBL1201823',
  4,
  None,
  None,
  None,
  None,
  None,
  None,
  None),
 ('ABCIXIMAB',
  None,
  None,
  'CHEMBL1201584',
  4,
  None,
  None,
  None,
  None,
  None,
  None,
  None),
 ('ADALIMUMAB',
  None,
  None,
  'CHEMBL1201580',
  4,
  None,
  None,
  None,
  None,
  None,
  None,
  None),
 ('AFLIBERCEPT',
  None,
  None,
  'CHEMBL1742982',
  4,
  None,
  None,
  None,
  None,
  None,
  None,
  None),
 ('AGALSIDASE ALFA',
  None,
  None,
  'CHEMBL2108214',
  4,
  None,
  None,
  None,
  None,
  None,
  None,
  None))

In [26]:
# Are the names unique? No, not same number of names as unique molregnos
print(len(set([i[0] for i in pref_name_result])) == len(set([i[1] for i in pref_name_result])))
print(len(set([i[0] for i in pref_name_result])), len(set([i[1] for i in pref_name_result])))

False
346 148


In [27]:
pref_name_result_df = pd.DataFrame(list(pref_name_result), columns = ['pref_name', 'standard_inchi', 'standard_inchi_key', 'chembl_id', 'max_phase', 'molregno', 'parent_molregno', 'parent_inchi', 'parent_inchi_key', 'parent_chembl_id', 'parent_max_phase', 'parent_pref_name'])

In [56]:
pref_name_result_df.loc[pref_name_result_df['parent_inchi'].isnull(),:]

,pref_name,standard_inchi,standard_inchi_key,chembl_id,max_phase,molregno,parent_molregno,parent_inchi,parent_inchi_key,parent_chembl_id,parent_max_phase,parent_pref_name
0,ABATACEPT,None,None,CHEMBL1201823,4,NaN,NaN,None,None,None,NaN,None
1,ABCIXIMAB,None,None,CHEMBL1201584,4,NaN,NaN,None,None,None,NaN,None
2,ADALIMUMAB,None,None,CHEMBL1201580,4,NaN,NaN,None,None,None,NaN,None
3,AFLIBERCEPT,None,None,CHEMBL1742982,4,NaN,NaN,None,None,None,NaN,None
4,AGALSIDASE ALFA,None,None,CHEMBL2108214,4,NaN,NaN,None,None,None,NaN,None
5,AGALSIDASE BETA,None,None,CHEMBL2108888,4,NaN,NaN,None,None,None,NaN,None
6,ALBIGLUTIDE,None,None,CHEMBL2107841,4,NaN,NaN,None,None,None,NaN,None
8,ALDESLEUKIN,None,None,CHEMBL1201438,4,NaN,NaN,None,None,None,NaN,None
9,ALEFACEPT,None,None,CHEMBL1201571,4,NaN,NaN,None,None,None,NaN,None
10,ALEMTUZUMAB,None,None,CHEMBL1201587,4,NaN,NaN,None,None,None,NaN,None


In [29]:
pref_name_counts = pref_name_result_df.groupby('pref_name').count()
names_count_result = [i for i in pref_name_counts.loc[pref_name_counts['molregno']>1,:].index]

In [30]:
names_count_result

['CETRIMIDE',
 'CHOLESTYRAMINE RESIN',
 'LITHIUM CITRATE',
 'MANGANESE CHLORIDE',
 'MANGANESE SULFATE',
 'MESNA',
 'PAPAVERETUM',
 'POTASSIUM SODIUM TARTRATE',
 'TALC',
 'TRYPAN BLUE',
 'ZINC ACETATE']

In [32]:
# For the ones with unequivocal mapping (count in names_count_query =1), and with parents, update mapping_dict
for row in pref_name_result_df.loc[~pref_name_result_df['parent_molregno'].isnull(),:].iterrows():
    rxnorm_name = row[1]['pref_name'] # mapped on pref_name
    if rxnorm_name.upper() in [i for i in names_count_result]:
        print('skipped {}'.format(rxnorm_name))
        continue
    for key in mapping_dict.keys():
        if mapping_dict[key]['rxnorm_name'].upper() == row[1]['pref_name']:
            mapping_dict[key]['parent_molregno']= row[1]['parent_molregno']
            mapping_dict[key]['parent_inchi']= row[1]['parent_inchi']
            mapping_dict[key]['parent_inchi_key']= row[1]['parent_inchi_key']
            mapping_dict[key]['parent_chembl_id']= row[1]['parent_chembl_id']
            mapping_dict[key]['parent_max_phase']= row[1]['parent_max_phase']
            mapping_dict[key]['mapping_notes'] = 'mapped RxNorm name on pref_name max_phase 4'

skipped CETRIMIDE
skipped CETRIMIDE
skipped CHOLESTYRAMINE RESIN
skipped LITHIUM CITRATE
skipped LITHIUM CITRATE
skipped MANGANESE CHLORIDE
skipped MANGANESE CHLORIDE
skipped MANGANESE SULFATE
skipped MESNA
skipped MESNA
skipped PAPAVERETUM
skipped POTASSIUM SODIUM TARTRATE
skipped TALC
skipped TRYPAN BLUE
skipped TRYPAN BLUE
skipped ZINC ACETATE
skipped ZINC ACETATE


In [44]:
# For the ones with unequivocal mapping (count in names_count_query =1), and without parents, update mapping_dict
for row in pref_name_result_df.loc[pref_name_result_df['parent_molregno'].isnull(),:].iterrows():
    rxnorm_name = row[1]['pref_name'] # mapped on pref_name
    if rxnorm_name.upper() in [i for i in names_count_result]:
        print('skipped {}'.format(rxnorm_name))
        continue
    for key in mapping_dict.keys():
        if mapping_dict[key]['rxnorm_name'].upper() == row[1]['pref_name']:
            mapping_dict[key]['parent_molregno']= row[1]['molregno']
            mapping_dict[key]['parent_inchi']= row[1]['standard_inchi']
            mapping_dict[key]['parent_inchi_key']= row[1]['standard_inchi_key']
            mapping_dict[key]['parent_chembl_id']= row[1]['chembl_id']
            mapping_dict[key]['parent_max_phase']= row[1]['max_phase']
            mapping_dict[key]['mapping_notes'] = 'mapped RxNorm name on pref_name max_phase 4'

skipped CHOLESTYRAMINE RESIN
skipped MANGANESE SULFATE
skipped PAPAVERETUM
skipped POTASSIUM SODIUM TARTRATE
skipped TALC


In [34]:
# For the ones with multiple mappings, do manual inspection
pref_name_result_df.loc[pref_name_result_df['pref_name'].isin(names_count_result),:].sort_values(by='pref_name')

,pref_name,standard_inchi,standard_inchi_key,chembl_id,max_phase,molregno,parent_molregno,parent_inchi,parent_inchi_key,parent_chembl_id,parent_max_phase,parent_pref_name
57,CETRIMIDE,InChI=1S/C17H38N.BrH/c1-5-6-7-8-9-10-11-12-13-...,CXRFDZFCGOPDTD-UHFFFAOYSA-M,CHEMBL113150,4,182325.0,653953.0,InChI=1S/C17H38N/c1-5-6-7-8-9-10-11-12-13-14-1...,GLFDLEXFOHUASB-UHFFFAOYSA-N,CHEMBL1180003,4.0,CETRIMIDE
58,CETRIMIDE,InChI=1S/C17H38N/c1-5-6-7-8-9-10-11-12-13-14-1...,GLFDLEXFOHUASB-UHFFFAOYSA-N,CHEMBL1180003,4,653953.0,653953.0,InChI=1S/C17H38N/c1-5-6-7-8-9-10-11-12-13-14-1...,GLFDLEXFOHUASB-UHFFFAOYSA-N,CHEMBL1180003,4.0,CETRIMIDE
62,CHOLESTYRAMINE RESIN,InChI=1S/C21H30N.ClH/c1-6-18-9-13-20(14-10-18)...,KNDHRUPPBXRELB-UHFFFAOYSA-M,CHEMBL2104535,4,1376805.0,1383073.0,InChI=1S/C21H30N/c1-6-18-9-13-20(14-10-18)17(2...,HGMSJMJPXGGEBP-UHFFFAOYSA-N,CHEMBL2110633,4.0,CHOLESTYRAMINE RESIN
63,CHOLESTYRAMINE RESIN,InChI=1S/C21H30N/c1-6-18-9-13-20(14-10-18)17(2...,HGMSJMJPXGGEBP-UHFFFAOYSA-N,CHEMBL2110633,4,1383073.0,NaN,None,None,None,NaN,None
173,LITHIUM CITRATE,"InChI=1S/C6H8O7.3Li/c7-3(8)1-6(13,5(11)12)2-4(...",WJSIUCDMWSDDCE-UHFFFAOYSA-K,CHEMBL1201170,4,675121.0,675121.0,"InChI=1S/C6H8O7.3Li/c7-3(8)1-6(13,5(11)12)2-4(...",WJSIUCDMWSDDCE-UHFFFAOYSA-K,CHEMBL1201170,4.0,LITHIUM CITRATE
174,LITHIUM CITRATE,"InChI=1S/C6H8O7.3Li.4H2O/c7-3(8)1-6(13,5(11)12...",HXGWMCJZLNWEBC-UHFFFAOYSA-K,CHEMBL2103738,4,1376008.0,675121.0,"InChI=1S/C6H8O7.3Li/c7-3(8)1-6(13,5(11)12)2-4(...",WJSIUCDMWSDDCE-UHFFFAOYSA-K,CHEMBL1201170,4.0,LITHIUM CITRATE
186,MANGANESE CHLORIDE,InChI=1S/2ClH.Mn.H2O/h2*1H;;1H2/q;;+2;/p-2,BXRRQHBNBXJZBQ-UHFFFAOYSA-L,CHEMBL1200548,4,674499.0,674644.0,InChI=1S/2ClH.Mn/h2*1H;/q;;+2/p-2,GLFNIEUTAYBVOC-UHFFFAOYSA-L,CHEMBL1200693,4.0,MANGANESE CHLORIDE
187,MANGANESE CHLORIDE,InChI=1S/2ClH.Mn/h2*1H;/q;;+2/p-2,GLFNIEUTAYBVOC-UHFFFAOYSA-L,CHEMBL1200693,4,674644.0,674644.0,InChI=1S/2ClH.Mn/h2*1H;/q;;+2/p-2,GLFNIEUTAYBVOC-UHFFFAOYSA-L,CHEMBL1200693,4.0,MANGANESE CHLORIDE
188,MANGANESE SULFATE,"InChI=1S/Mn.H2O4S/c;1-5(2,3)4/h;(H2,1,2,3,4)/q...",SQQMAOCOWKFBNP-UHFFFAOYSA-L,CHEMBL1200557,4,674508.0,NaN,None,None,None,NaN,None
189,MANGANESE SULFATE,"InChI=1S/Mn.H2O4S.H2O/c;1-5(2,3)4;/h;(H2,1,2,3...",ISPYRSDWRDQNSW-UHFFFAOYSA-L,CHEMBL2103742,4,1376012.0,674508.0,"InChI=1S/Mn.H2O4S/c;1-5(2,3)4/h;(H2,1,2,3,4)/q...",SQQMAOCOWKFBNP-UHFFFAOYSA-L,CHEMBL1200557,4.0,MANGANESE SULFATE


In [45]:
# In these cases, picked parent molecule.
# In a few cases the parent seems automatically generated because it is not in compound records and is not its own parent in molecule_hierarchy
# Looks like such automatically generated parents have max_phase = 4 assigned

# cetrimide parent: CHEMBL1180003
# CHOLESTYRAMINE RESIN: CHEMBL2110633
# LITHIUM CITRATE: CHEMBL1201170
# MANGANESE CHLORIDE: CHEMBL1200693
# MANGANESE SULFATE: CHEMBL1200557
# MESNA: CHEMBL1098319
# papaveretum parent: CHEMBL3833406
# POTASSIUM SODIUM TARTRATE: CHEMBL2219738
# TALC: CHEMBL3990276
# TRYPAN BLUE: CHEMBL1089641
# ZINC ACETATE: CHEMBL1200928


In [46]:
manual_names_query1 = """
select distinct md.molregno, md.chembl_id, md.pref_name, str.standard_inchi, str.standard_inchi_key
from molecule_dictionary md join compound_structures str on md.molregno = str.molregno
where md.chembl_id in ('CHEMBL1180003', 'CHEMBL2110633', 'CHEMBL1201170', 'CHEMBL1200693', 'CHEMBL1200557', 'CHEMBL1098319', 'CHEMBL3833406', 'CHEMBL2219738', 'CHEMBL3990276', 'CHEMBL1089641', 'CHEMBL1200928')
and md.max_phase = 4
"""
manual_names_query1_result = do_query(manual_names_query1)

In [47]:
manual_names_query1_result_df = pd.DataFrame(list(manual_names_query1_result), columns = ['parent_molregno','parent_chembl_id', 'parent_pref_name', 'parent_inchi', 'parent_inchi_key'])

In [48]:
manual_names_query1_result_df

,parent_molregno,parent_chembl_id,parent_pref_name,parent_inchi,parent_inchi_key
0,622436,CHEMBL1089641,TRYPAN BLUE,InChI=1S/C34H28N6O14S4/c1-15-7-17(3-5-25(15)37...,ZBNARPCCDMHDDV-HVMBLDELSA-N
1,624467,CHEMBL1098319,MESNA,"InChI=1S/C2H6O3S2/c3-7(4,5)2-1-6/h6H,1-2H2,(H,...",ZNEWHQLOPFWXOF-UHFFFAOYSA-N
2,653953,CHEMBL1180003,CETRIMIDE,InChI=1S/C17H38N/c1-5-6-7-8-9-10-11-12-13-14-1...,GLFDLEXFOHUASB-UHFFFAOYSA-N
3,674508,CHEMBL1200557,MANGANESE SULFATE,"InChI=1S/Mn.H2O4S/c;1-5(2,3)4/h;(H2,1,2,3,4)/q...",SQQMAOCOWKFBNP-UHFFFAOYSA-L
4,674644,CHEMBL1200693,MANGANESE CHLORIDE,InChI=1S/2ClH.Mn/h2*1H;/q;;+2/p-2,GLFNIEUTAYBVOC-UHFFFAOYSA-L
5,674879,CHEMBL1200928,ZINC ACETATE,"InChI=1S/2C2H4O2.Zn/c2*1-2(3)4;/h2*1H3,(H,3,4)...",DJWUNCQRNNEAKC-UHFFFAOYSA-L
6,675121,CHEMBL1201170,LITHIUM CITRATE,"InChI=1S/C6H8O7.3Li/c7-3(8)1-6(13,5(11)12)2-4(...",WJSIUCDMWSDDCE-UHFFFAOYSA-K
7,1383073,CHEMBL2110633,CHOLESTYRAMINE RESIN,InChI=1S/C21H30N/c1-6-18-9-13-20(14-10-18)17(2...,HGMSJMJPXGGEBP-UHFFFAOYSA-N
8,1449549,CHEMBL2219738,POTASSIUM SODIUM TARTRATE,InChI=1S/C4H6O6.K.Na/c5-1(3(7)8)2(6)4(9)10;;/h...,LJCNRYVRMXRIQR-OLXYHTOASA-L
9,2096457,CHEMBL3833406,PAPAVERETUM,InChI=1S/C20H21NO4.C18H21NO3.C17H19NO3/c1-22-1...,IAWXESBTFLIKNX-MMWHECRDSA-N


In [49]:
# Update dictionary for these manually checked compounds
for row in manual_names_query1_result_df.iterrows():
    for key in mapping_dict.keys():
        if mapping_dict[key]['rxnorm_name'].upper() == row[1]['parent_pref_name']:
            mapping_dict[key]['parent_molregno']= row[1]['parent_molregno']
            mapping_dict[key]['parent_inchi']= row[1]['parent_inchi']
            mapping_dict[key]['parent_inchi_key']= row[1]['parent_inchi_key']
            mapping_dict[key]['parent_chembl_id']= row[1]['parent_chembl_id']
            mapping_dict[key]['parent_max_phase']= 4
            mapping_dict[key]['mapping_notes'] = 'mapped rxnorm name to chembl pref_name, max_phase 4, manual inspection chose parent compound'

In [55]:
# Handy bit check mapped
still_unassigned = []
for key in mapping_dict.keys():
    try:
        mapping_dict[key]['parent_chembl_id']
    except KeyError:
        still_unassigned.append(mapping_dict[key]['rxnorm_concept'])
print(len(still_unassigned))

1181


In [51]:
#### Tier 2 - compound name ###

In [57]:
# Make string of RxNorm drug names currently unmapped
still_unassigned_ids = []
for key in mapping_dict.keys():
    try:
        mapping_dict[key]['parent_chembl_id']
    except KeyError:
        still_unassigned_ids.append(key)
joined_names_query2 = ', '.join(['"'+j+'"' for j in [mapping_dict[key]['rxnorm_name'].upper() for key in still_unassigned_ids] if (not '(' in j and not ')' in j)])    

In [58]:
len(still_unassigned_ids)

1181

In [59]:
# Query 2 - RxNorm name to ChEMBL compound_name
compound_name_query = """select distinct 
md.chembl_id
, md.pref_name
, cr.compound_name
, str.standard_inchi
, str.standard_inchi_key
, md.molregno
, hier.parent_molregno
, md2.pref_name as parent_pref_name
, md2.chembl_id as parent_chembl_id
, str2.standard_inchi as parent_inchi
, str2.standard_inchi_key as parent_inchi_key
from compound_records cr 
join molecule_dictionary md on md.molregno = cr.molregno
left join compound_structures str on str.molregno = md.molregno
left join molecule_hierarchy hier on hier.molregno = md.molregno
left join compound_structures str2 on str2.molregno = hier.parent_molregno
left join molecule_dictionary md2 on md2.molregno = hier.parent_molregno
where cr.compound_name in ({})
and md.max_phase = 4
;""".format(joined_names_query2)

In [60]:
query2_result = do_query(compound_name_query)

In [61]:
len(query2_result)

30

In [62]:
query2_result_df = pd.DataFrame(list(query2_result), columns = ['chembl_id', 'pref_name', 'compound_name', 'standard_inchi', 'standard_inchi_key', 'molregno', 'parent_molregno', 'parent_pref_name', 'parent_chembl_id', 'parent_inchi', 'parent_inchi_key'])

In [63]:
query2_result_df.loc[query2_result_df['parent_molregno'].isnull(),:]

,chembl_id,pref_name,compound_name,standard_inchi,standard_inchi_key,molregno,parent_molregno,parent_pref_name,parent_chembl_id,parent_inchi,parent_inchi_key


In [64]:
name_counts = query2_result_df.groupby('compound_name').count()
names_count_result = [i for i in name_counts.loc[name_counts['molregno']>1,:].index]

In [65]:
names_count_result

[]

In [66]:
# All of the mappings are unique, did manual inspection
# tried to find 'tadenan' in the original paper but couldn't find it. Accept all of these except Tadenan.

In [67]:
#  update mapping_dict
for row in query2_result_df.iterrows():
    rxnorm_name = row[1]['compound_name'] # mapped on compound_name
    if rxnorm_name.upper() == 'TADENAN':
        print('skipped {}'.format(rxnorm_name))
        continue
    for key in mapping_dict.keys():
        if mapping_dict[key]['rxnorm_name'].upper() == row[1]['compound_name'].upper():
            mapping_dict[key]['parent_molregno']= row[1]['parent_molregno']
            mapping_dict[key]['parent_inchi']= row[1]['parent_inchi']
            mapping_dict[key]['parent_inchi_key']= row[1]['parent_inchi_key']
            mapping_dict[key]['parent_chembl_id']= row[1]['parent_chembl_id']
            mapping_dict[key]['parent_max_phase']= 4
            mapping_dict[key]['mapping_notes'] = 'mapped RxNorm name on compound_name max_phase 4'

skipped tadenan


In [68]:
# Handy bit check mapped
still_unassigned = []
for key in mapping_dict.keys():
    try:
        mapping_dict[key]['parent_inchi_key']
    except KeyError:
        still_unassigned.append(mapping_dict[key]['rxnorm_concept'])
print(len(still_unassigned))

1152


In [69]:
# Handy bit check mapped
for key in mapping_dict.keys():
    try:
        mapping_dict[key]['parent_inchi_key']
        if mapping_dict[key]['mapping_notes']=='mapped RxNorm name on compound_name max_phase 4':
            print(mapping_dict[key])
    except KeyError:
        continue

{'aeolus_concept': 711452, 'rxnorm_concept': 70589, 'rxnorm_name': 'Benzoate', 'drugbank_id': None, 'parent_molregno': 13371, 'parent_inchi': 'InChI=1S/C7H6O2/c8-7(9)6-4-2-1-3-5-6/h1-5H,(H,8,9)', 'parent_inchi_key': 'WPYMKLBDIGXBTP-UHFFFAOYSA-N', 'parent_chembl_id': 'CHEMBL541', 'parent_max_phase': 4, 'mapping_notes': 'mapped RxNorm name on compound_name max_phase 4'}
{'aeolus_concept': 952045, 'rxnorm_concept': 42527, 'rxnorm_name': 'Amphotericin', 'drugbank_id': None, 'parent_molregno': 8873, 'parent_inchi': 'InChI=1S/C47H73NO17/c1-27-17-15-13-11-9-7-5-6-8-10-12-14-16-18-34(64-46-44(58)41(48)43(57)30(4)63-46)24-38-40(45(59)60)37(54)26-47(61,65-38)25-33(51)22-36(53)35(52)20-19-31(49)21-32(50)23-39(55)62-29(3)28(2)42(27)56/h5-18,27-38,40-44,46,49-54,56-58,61H,19-26,48H2,1-4H3,(H,59,60)/b6-5+,9-7+,10-8+,13-11+,14-12+,17-15+,18-16+/t27-,28-,29-,30+,31+,32+,33-,34-,35+,36+,37-,38-,40+,41-,42+,43+,44-,46-,47+/m0/s1', 'parent_inchi_key': 'APKFDSVGJQXUKY-INPOYWNPSA-N', 'parent_chembl_id': 'C

In [70]:
# Query 3 - RxNorm name to ChEMBL synonyms

In [71]:
# Make string of RxNorm drug names currently unmapped
still_unassigned_ids = []
for key in mapping_dict.keys():
    try:
        mapping_dict[key]['parent_inchi_key']
    except KeyError:
        still_unassigned_ids.append(key)
joined_names_query3 = ', '.join(['"'+j+'"' for j in [mapping_dict[key]['rxnorm_name'].upper() for key in still_unassigned_ids] if (not '(' in j and not ')' in j)])    

In [72]:
len(still_unassigned_ids)

1152

In [73]:
synonyms_query3 = """
select distinct 
md.chembl_id
, md.pref_name
, syn.synonyms
, str.standard_inchi
, str.standard_inchi_key
, md.molregno
, hier.parent_molregno
, md2.pref_name as parent_pref_name
, md2.chembl_id as parent_chembl_id
, str2.standard_inchi as parent_inchi
, str2.standard_inchi_key as parent_inchi_key
from molecule_dictionary md
join molecule_synonyms syn on md.molregno = syn.molregno
left join compound_structures str on str.molregno = md.molregno
left join molecule_hierarchy hier on hier.molregno = md.molregno
left join compound_structures str2 on str2.molregno = hier.parent_molregno
left join molecule_dictionary md2 on md2.molregno = hier.parent_molregno
where syn.synonyms in ({0})
and md.max_phase = 4
and syn.syn_type != 'TRADE_NAME'
""".format(joined_names_query3)

In [74]:
query3_result = do_query(synonyms_query3)

In [75]:
query3_result_df = pd.DataFrame(list(query3_result), columns = ['chembl_id', 'pref_name','synonyms', 'standard_inchi', 'standard_inchi_key', 'molregno', 'parent_molregno', 'parent_pref_name', 'parent_chembl_id', 'parent_inchi', 'parent_inchi_key'])

In [76]:
query3_result_df.loc[query3_result_df['parent_molregno'].isnull(),:]

,chembl_id,pref_name,synonyms,standard_inchi,standard_inchi_key,molregno,parent_molregno,parent_pref_name,parent_chembl_id,parent_inchi,parent_inchi_key


In [77]:
# Manual inspection, I'm unsure about Pristinamycin and Follicle Stimulating Hormone, seem to be multiple component concepts

In [78]:
name_counts = query3_result_df.groupby('synonyms').count()
names_count_result = [i for i in name_counts.loc[name_counts['molregno']>1,:].index]

In [79]:
names_count_result

[]

In [80]:
#  update mapping_dict
for row in query3_result_df.iterrows():
    rxnorm_name = row[1]['synonyms'] # mapped on synonyms
    if rxnorm_name.upper() in ['Pristinamycin'.upper(), 'Follicle Stimulating Hormone'.upper()]:
        print('skipped {}'.format(rxnorm_name))
        continue
    for key in mapping_dict.keys():
        if mapping_dict[key]['rxnorm_name'].upper() == row[1]['synonyms'].upper():
            mapping_dict[key]['parent_molregno']= row[1]['parent_molregno']
            mapping_dict[key]['parent_inchi']= row[1]['parent_inchi']
            mapping_dict[key]['parent_inchi_key']= row[1]['parent_inchi_key']
            mapping_dict[key]['parent_chembl_id']= row[1]['parent_chembl_id']
            mapping_dict[key]['parent_max_phase']= 4
            mapping_dict[key]['mapping_notes'] = 'mapped RxNorm name on chembl synonyms max_phase 4'

skipped Follicle Stimulating Hormone
skipped Pristinamycin


In [81]:
# Handy bit check mapped
still_unassigned = []
for key in mapping_dict.keys():
    try:
        mapping_dict[key]['parent_chembl_id']
    except KeyError:
        still_unassigned.append(mapping_dict[key]['rxnorm_concept'])
print(len(still_unassigned))

1116


In [82]:
### Query 4, RxNorm name against pref_name, not max_phase = 4 #####

In [85]:
# Make string of RxNorm drug names currently unmapped
still_unassigned_ids = []
for key in mapping_dict.keys():
    try:
        mapping_dict[key]['parent_chembl_id']
    except KeyError:
        still_unassigned_ids.append(key)
joined_names_query4 = ', '.join(['"'+j+'"' for j in [mapping_dict[key]['rxnorm_name'].upper() for key in still_unassigned_ids] if (not '(' in j and not ')' in j)])    

In [86]:
len(still_unassigned_ids)

1116

In [87]:
# RxNorm name to ChEMBL pref_name, not max_phase = 4
pref_name_query_allphases = """select distinct
md.pref_name
, str.standard_inchi
, str.standard_inchi_key
, md.chembl_id
, md.max_phase
, str.molregno
, hier.parent_molregno
, str2.standard_inchi as parent_inchi
, str2.standard_inchi_key as parent_inchi_key
, md2.chembl_id as parent_chembl_id
, md2.max_phase as parent_max_phase
, md2.pref_name as parent_pref_name
from molecule_dictionary md 
left join compound_structures str on md.molregno = str.molregno
left join molecule_hierarchy hier on hier.molregno = str.molregno
left join compound_structures str2 on str2.molregno = hier.parent_molregno
left join molecule_dictionary md2 on md2.molregno = hier.parent_molregno
where md.pref_name in ({})
and md.max_phase != 4""".format(joined_names_query4)

In [88]:
query4_result = do_query(pref_name_query_allphases)

In [89]:
len(query4_result)

373

In [90]:
query4_result_df = pd.DataFrame(list(query4_result), columns = ['pref_name', 'standard_inchi', 'standard_inchi_key', 'chembl_id', 'max_phase', 'molregno', 'parent_molregno', 'parent_inchi', 'parent_inchi_key', 'parent_chembl_id', 'parent_max_phase', 'parent_pref_name'])

In [91]:
query4_result_df.loc[query4_result_df['parent_molregno'].isnull(),:]

,pref_name,standard_inchi,standard_inchi_key,chembl_id,max_phase,molregno,parent_molregno,parent_inchi,parent_inchi_key,parent_chembl_id,parent_max_phase,parent_pref_name
11,ALGINIC ACID,None,None,CHEMBL2108894,3,NaN,NaN,None,None,None,NaN,None
15,ALMOND OIL,None,None,CHEMBL2108220,0,NaN,NaN,None,None,None,NaN,None
17,ALOGLUTAMOL,InChI=1S/C6H12O7.Al.2H2O/c7-1-2(8)3(9)4(10)5(1...,YMILRBLJDGKUDX-CXKLNRRHSA-K,CHEMBL3707300,0,2039271.0,NaN,None,None,None,NaN,None
20,ALUMINUM ZIRCONIUM TETRACHLOROHYDREX GLY,None,None,CHEMBL2108844,0,NaN,NaN,None,None,None,NaN,None
21,ALUMINUM ZIRCONIUM TRICHLOROHYDREX GLY,None,None,CHEMBL2109022,0,NaN,NaN,None,None,None,NaN,None
22,AMEZINIUM,InChI=1S/C11H11N3O/c1-15-11-7-9(12)8-13-14(11)...,VXROHTDSRBRJLN-UHFFFAOYSA-O,CHEMBL2110922,0,1383362.0,NaN,None,None,None,NaN,None
23,AMILOMER,None,None,CHEMBL2108362,0,NaN,NaN,None,None,None,NaN,None
27,ANCESTIM,None,None,CHEMBL2108975,2,NaN,NaN,None,None,None,NaN,None
28,ANCROD,None,None,CHEMBL2108481,3,NaN,NaN,None,None,None,NaN,None
32,ANISE OIL,None,None,CHEMBL2108976,0,NaN,NaN,None,None,None,NaN,None


In [92]:
pref_name_counts = query4_result_df.groupby('pref_name').count()
names_count_result = [i for i in pref_name_counts.loc[pref_name_counts['molregno']>1,:].index]

In [93]:
names_count_result

['ALOGLUTAMOL',
 'BUTYLATED HYDROXYANISOLE',
 'FELYPRESSIN',
 'MAGNESIUM PHOSPHATE',
 'PAMABROM',
 'POTASSIUM ASPARTATE',
 'SODIUM PROPIONATE',
 'THIOSTREPTON']

In [94]:
# For the ones with unequivocal mapping (count in names_count_query =1), with parents, update mapping_dict
for row in query4_result_df.loc[~query4_result_df['parent_molregno'].isnull(),:].iterrows():
    rxnorm_name = row[1]['pref_name'] # mapped on pref_name
    if rxnorm_name.upper() in [i for i in names_count_result]:
        print('skipped {}'.format(rxnorm_name))
        continue
    for key in mapping_dict.keys():
        if mapping_dict[key]['rxnorm_name'].upper() == row[1]['pref_name']:
            mapping_dict[key]['parent_molregno']= row[1]['parent_molregno']
            mapping_dict[key]['parent_inchi']= row[1]['parent_inchi']
            mapping_dict[key]['parent_inchi_key']= row[1]['parent_inchi_key']
            mapping_dict[key]['parent_chembl_id']= row[1]['parent_chembl_id']
            mapping_dict[key]['parent_max_phase']= row[1]['parent_max_phase']
            mapping_dict[key]['mapping_notes'] = 'mapped RxNorm name on pref_name max_phase != 4'

skipped ALOGLUTAMOL
skipped BUTYLATED HYDROXYANISOLE
skipped BUTYLATED HYDROXYANISOLE
skipped FELYPRESSIN
skipped FELYPRESSIN
skipped MAGNESIUM PHOSPHATE
skipped MAGNESIUM PHOSPHATE
skipped PAMABROM
skipped PAMABROM
skipped POTASSIUM ASPARTATE
skipped SODIUM PROPIONATE
skipped SODIUM PROPIONATE
skipped THIOSTREPTON
skipped THIOSTREPTON


In [95]:
# For the ones with unequivocal mapping (count in names_count_query =1), without parents, update mapping_dict
for row in query4_result_df.loc[query4_result_df['parent_molregno'].isnull(),:].iterrows():
    rxnorm_name = row[1]['pref_name'] # mapped on pref_name
    if rxnorm_name.upper() in [i for i in names_count_result]:
        print('skipped {}'.format(rxnorm_name))
        continue
    for key in mapping_dict.keys():
        if mapping_dict[key]['rxnorm_name'].upper() == row[1]['pref_name']:
            mapping_dict[key]['parent_molregno']= row[1]['molregno']
            mapping_dict[key]['parent_inchi']= row[1]['standard_inchi']
            mapping_dict[key]['parent_inchi_key']= row[1]['standard_inchi_key']
            mapping_dict[key]['parent_chembl_id']= row[1]['chembl_id']
            mapping_dict[key]['parent_max_phase']= row[1]['max_phase']
            mapping_dict[key]['mapping_notes'] = 'mapped RxNorm name on pref_name max_phase != 4'

skipped ALOGLUTAMOL
skipped POTASSIUM ASPARTATE


In [96]:
# Now look at the ones with more than one row
query4_result_df.loc[query4_result_df['pref_name'].isin(names_count_result),:]

,pref_name,standard_inchi,standard_inchi_key,chembl_id,max_phase,molregno,parent_molregno,parent_inchi,parent_inchi_key,parent_chembl_id,parent_max_phase,parent_pref_name
16,ALOGLUTAMOL,InChI=1S/C6H12O7.C4H11NO3.Al.2H2O/c7-1-2(8)3(9...,GJJYZOBRHIMORS-GQOAHPRESA-K,CHEMBL3707299,0,2039270.0,2039271.0,InChI=1S/C6H12O7.Al.2H2O/c7-1-2(8)3(9)4(10)5(1...,YMILRBLJDGKUDX-CXKLNRRHSA-K,CHEMBL3707300,0.0,ALOGLUTAMOL
17,ALOGLUTAMOL,InChI=1S/C6H12O7.Al.2H2O/c7-1-2(8)3(9)4(10)5(1...,YMILRBLJDGKUDX-CXKLNRRHSA-K,CHEMBL3707300,0,2039271.0,NaN,None,None,None,NaN,None
65,BUTYLATED HYDROXYANISOLE,"InChI=1S/C11H16O2/c1-11(2,3)9-7-8(13-4)5-6-10(...",MRBKEAMVRSLQPH-UHFFFAOYSA-N,CHEMBL192451,0,324645.0,324645.0,"InChI=1S/C11H16O2/c1-11(2,3)9-7-8(13-4)5-6-10(...",MRBKEAMVRSLQPH-UHFFFAOYSA-N,CHEMBL192451,0.0,BUTYLATED HYDROXYANISOLE
66,BUTYLATED HYDROXYANISOLE,"InChI=1S/C11H16O2/c1-11(2,3)9-7-8(12)5-6-10(9)...",IMOYOUMVYICGCA-UHFFFAOYSA-N,CHEMBL502074,0,461652.0,461652.0,"InChI=1S/C11H16O2/c1-11(2,3)9-7-8(12)5-6-10(9)...",IMOYOUMVYICGCA-UHFFFAOYSA-N,CHEMBL502074,0.0,BUTYLATED HYDROXYANISOLE
147,FELYPRESSIN,InChI=1S/C47H66N12O11S2/c48-18-8-7-14-29(41(64...,JEFCJMMLFXHSSX-AVIVMPHOSA-N,CHEMBL1908309,0,1248680.0,1248680.0,InChI=1S/C47H66N12O11S2/c48-18-8-7-14-29(41(64...,JEFCJMMLFXHSSX-AVIVMPHOSA-N,CHEMBL1908309,0.0,FELYPRESSIN
148,FELYPRESSIN,InChI=1S/C46H65N13O11S2/c47-18-8-7-14-29(40(64...,SFKQVVDKFKYTNA-OGNLOKFRSA-N,CHEMBL3989781,0,2197652.0,2197652.0,InChI=1S/C46H65N13O11S2/c47-18-8-7-14-29(40(64...,SFKQVVDKFKYTNA-OGNLOKFRSA-N,CHEMBL3989781,0.0,FELYPRESSIN
212,MAGNESIUM PHOSPHATE,"InChI=1S/3Mg.2H3O4P/c;;;2*1-5(2,3)4/h;;;2*(H3,...",GVALZJMUIHGIMD-UHFFFAOYSA-H,CHEMBL2106873,0,1379143.0,1379143.0,"InChI=1S/3Mg.2H3O4P/c;;;2*1-5(2,3)4/h;;;2*(H3,...",GVALZJMUIHGIMD-UHFFFAOYSA-H,CHEMBL2106873,0.0,MAGNESIUM PHOSPHATE
213,MAGNESIUM PHOSPHATE,"InChI=1S/3Mg.2H3O4P.5H2O/c;;;2*1-5(2,3)4;;;;;/...",QZYXGKYJDBUYES-UHFFFAOYSA-H,CHEMBL3989627,0,2197498.0,1379143.0,"InChI=1S/3Mg.2H3O4P/c;;;2*1-5(2,3)4/h;;;2*(H3,...",GVALZJMUIHGIMD-UHFFFAOYSA-H,CHEMBL2106873,0.0,MAGNESIUM PHOSPHATE
260,PAMABROM,InChI=1S/C7H7BrN4O2/c1-11-4-3(9-6(8)10-4)5(13)...,SKTFQHRVFFOHTQ-UHFFFAOYSA-N,CHEMBL316160,0,143943.0,143943.0,InChI=1S/C7H7BrN4O2/c1-11-4-3(9-6(8)10-4)5(13)...,SKTFQHRVFFOHTQ-UHFFFAOYSA-N,CHEMBL316160,0.0,PAMABROM
261,PAMABROM,InChI=1S/C7H7BrN4O2.C4H11NO/c1-11-4-3(9-6(8)10...,ATOTUUBRFJHZQG-UHFFFAOYSA-N,CHEMBL2104825,0,1377095.0,143943.0,InChI=1S/C7H7BrN4O2/c1-11-4-3(9-6(8)10-4)5(13)...,SKTFQHRVFFOHTQ-UHFFFAOYSA-N,CHEMBL316160,0.0,PAMABROM


In [97]:
# Pick the parents or compound with more activities in ChEMBL - manual inspection

In [98]:
manual_names_query2 = """
select distinct md.max_phase, md.molregno, md.chembl_id, md.pref_name, str.standard_inchi, str.standard_inchi_key
from molecule_dictionary md 
left join compound_structures str on md.molregno = str.molregno
where md.chembl_id in ('CHEMBL3707300', 'CHEMBL502074', 'CHEMBL1908309', 'CHEMBL2106873', 'CHEMBL316160', 'CHEMBL3990695', 'CHEMBL500826', 'CHEMBL2303629')
"""
manual_names_query2_result = do_query(manual_names_query2)

In [99]:
manual_names_query2_result_df = pd.DataFrame(list(manual_names_query2_result), columns = ['max_phase','molregno','chembl_id', 'pref_name', 'standard_inchi', 'standard_inchi_key'])

In [100]:
manual_names_query2_result_df

,max_phase,molregno,chembl_id,pref_name,standard_inchi,standard_inchi_key
0,0,1248680,CHEMBL1908309,FELYPRESSIN,InChI=1S/C47H66N12O11S2/c48-18-8-7-14-29(41(64...,JEFCJMMLFXHSSX-AVIVMPHOSA-N
1,0,1379143,CHEMBL2106873,MAGNESIUM PHOSPHATE,"InChI=1S/3Mg.2H3O4P/c;;;2*1-5(2,3)4/h;;;2*(H3,...",GVALZJMUIHGIMD-UHFFFAOYSA-H
2,0,1503100,CHEMBL2303629,THIOSTREPTON,InChI=1S/C72H85N19O18S5/c1-14-26(3)47-63(105)7...,NSFFHOGKXHRQEW-BEEVBQKTSA-N
3,0,143943,CHEMBL316160,PAMABROM,InChI=1S/C7H7BrN4O2/c1-11-4-3(9-6(8)10-4)5(13)...,SKTFQHRVFFOHTQ-UHFFFAOYSA-N
4,0,2039271,CHEMBL3707300,ALOGLUTAMOL,InChI=1S/C6H12O7.Al.2H2O/c7-1-2(8)3(9)4(10)5(1...,YMILRBLJDGKUDX-CXKLNRRHSA-K
5,0,2198566,CHEMBL3990695,POTASSIUM ASPARTATE,InChI=1S/2C4H7NO4.2K/c2*5-2(4(8)9)1-3(6)7;;/h2...,AJDJWPBAUNUEOB-NVKWYWNSSA-L
6,2,476867,CHEMBL500826,SODIUM PROPIONATE,"InChI=1S/C3H6O2.Na/c1-2-3(4)5;/h2H2,1H3,(H,4,5...",JXKPEJDQGNYQSM-UHFFFAOYSA-M
7,0,461652,CHEMBL502074,BUTYLATED HYDROXYANISOLE,"InChI=1S/C11H16O2/c1-11(2,3)9-7-8(12)5-6-10(9)...",IMOYOUMVYICGCA-UHFFFAOYSA-N


In [101]:
# Update dictionary for these manually checked compounds
for row in manual_names_query2_result_df.iterrows():
    for key in mapping_dict.keys():
        try:
            mapping_dict[key]['parent_inchi_key']
            continue
        except KeyError:
            if mapping_dict[key]['rxnorm_name'].upper() == row[1]['pref_name']:
                mapping_dict[key]['parent_molregno']= row[1]['molregno']
                mapping_dict[key]['parent_inchi']= row[1]['standard_inchi']
                mapping_dict[key]['parent_inchi_key']= row[1]['standard_inchi_key']
                mapping_dict[key]['parent_chembl_id']= row[1]['chembl_id']
                mapping_dict[key]['parent_max_phase']= row[1]['max_phase']
                mapping_dict[key]['mapping_notes'] = 'mapped rxnorm name to chembl pref_name, not max_phase 4, manual inspection chose parent compound or with most activities'

In [102]:
# Handy bit check mapped
still_unassigned = []
for key in mapping_dict.keys():
    try:
        mapping_dict[key]['parent_chembl_id']
    except KeyError:
        still_unassigned.append(mapping_dict[key]['rxnorm_concept'])
print(len(still_unassigned))

752


In [103]:
##### Query 5: RxNorm MTHSPL (structured product label approved by FDA) synonym against ChEMBL pref_name, not max_phase = 4 #####

In [104]:
# Make string of RxNorm drug names currently unmapped
still_unassigned_ids = []
for key in mapping_dict.keys():
    try:
        mapping_dict[key]['parent_inchi_key']
    except KeyError:
        still_unassigned_ids.append(key)
joined_names_query5 = ', '.join(['"'+j+'"' for j in [mapping_dict[key]['rxnorm_name'].upper() for key in still_unassigned_ids] if (not '(' in j and not ')' in j)])    

In [105]:
len(still_unassigned_ids)

752

In [106]:
level5_unmapped_rxnorm_ids = [mapping_dict[key]['rxnorm_concept'] for key in still_unassigned_ids]
print(len(level5_unmapped_rxnorm_ids))

752


In [107]:
# File RXNCONSO provided by RxNorm
with open('/Users/ines/PycharmProjects/FAERS_y2/data/raw/RxNorm_full_12032018/rrf/RXNCONSO.RRF') as f:
    data = f.readlines()

In [108]:
# Load a dictionary with the rxnorm concepts currently unassigned, this adds all synonyms as dict per rxnorm id, with the synonym type/vocabulary as the key to the synonym value
# Takes a few minutes

rxnorm_concepts = dict()
for rxnorm_id in level5_unmapped_rxnorm_ids:
    rxnorm_concepts[rxnorm_id] = {}

for row in data:
    elements = row.strip('\n').split('|')
    if int(elements[0]) in level5_unmapped_rxnorm_ids:
        vocab = elements[-8]
        rxnorm_concepts[int(elements[0])][vocab] = elements[-5]
    else:
        continue

In [109]:
len(rxnorm_concepts.keys())

743

In [110]:
# First try using SPL names, query ChEMBL synonyms using the SPL names
spl_concepts = []
spl_names = []
for concept in rxnorm_concepts.keys():
    try:
        spl_synonym = rxnorm_concepts[concept]['MTHSPL']
        if (not '(' in spl_synonym and not ')' in spl_synonym and not "'" in spl_synonym):
            spl_names.append(spl_synonym)
            spl_concepts.append((spl_synonym, concept))
    except KeyError:
        continue

In [111]:
# RxNorm name to ChEMBL pref_name, any phase
spl_names_pref_name_query = """select distinct
md.pref_name
, str.standard_inchi
, str.standard_inchi_key
, md.chembl_id
, md.max_phase
, md.molregno
, hier.parent_molregno
, str2.standard_inchi as parent_inchi
, str2.standard_inchi_key as parent_inchi_key
, md2.chembl_id as parent_chembl_id
, md2.max_phase as parent_max_phase
, md2.pref_name as parent_pref_name
from molecule_dictionary md 
left join compound_structures str on md.molregno = str.molregno
left join molecule_hierarchy hier on hier.molregno = str.molregno
left join compound_structures str2 on str2.molregno = hier.parent_molregno
left join molecule_dictionary md2 on md2.molregno = hier.parent_molregno
where md.pref_name in ({})""".format(', '.join(['"'+str(j)+'"' for j in spl_names]))

In [112]:
spl_names_pref_name_result = do_query(spl_names_pref_name_query)

In [113]:
len(spl_names_pref_name_result)

38

In [114]:
spl_names_pref_name_result_df = pd.DataFrame(list(spl_names_pref_name_result), columns = ['pref_name', 'standard_inchi', 'standard_inchi_key', 'chembl_id', 'max_phase', 'molregno', 'parent_molregno', 'parent_inchi', 'parent_inchi_key', 'parent_chembl_id', 'parent_max_phase', 'parent_pref_name'])

In [115]:
pref_name_counts = spl_names_pref_name_result_df.groupby('pref_name').count()
names_count_result = [i for i in pref_name_counts.loc[pref_name_counts['molregno']>1,:].index]
names_count_result

['PYRITHIONE ZINC', 'TEPRENONE']

In [116]:
# For PYRITHIONE ZINC', 'TEPRENONE', pick compound with highest phase

In [117]:
# drop CHEMBL79686, CHEMBL1200471
spl_names_pref_name_result_df = spl_names_pref_name_result_df.loc[~spl_names_pref_name_result_df['chembl_id'].isin(['CHEMBL79686','CHEMBL1200471']),:]

In [118]:
len(spl_names_pref_name_result_df)

36

In [119]:
spl_names_pref_name_result_df.loc[~spl_names_pref_name_result_df['parent_molregno'].isnull(),:]

,pref_name,standard_inchi,standard_inchi_key,chembl_id,max_phase,molregno,parent_molregno,parent_inchi,parent_inchi_key,parent_chembl_id,parent_max_phase,parent_pref_name
9,GAMMA ORYZANOL,InChI=1S/C40H58O4/c1-26(2)10-9-11-27(3)29-18-2...,FODTZLFLDFKIQH-FSVGXZBPSA-N,CHEMBL388595,0,379107,379107.0,InChI=1S/C40H58O4/c1-26(2)10-9-11-27(3)29-18-2...,FODTZLFLDFKIQH-FSVGXZBPSA-N,CHEMBL388595,0.0,GAMMA ORYZANOL
13,GLUCUROLACTONE,InChI=1S/C6H8O6/c7-1-3-4(12-5(1)9)2(8)6(10)11-...,OGLCQHRZUSEXNB-UAPNVWQMSA-N,CHEMBL2107425,0,1379695,1379695.0,InChI=1S/C6H8O6/c7-1-3-4(12-5(1)9)2(8)6(10)11-...,OGLCQHRZUSEXNB-UAPNVWQMSA-N,CHEMBL2107425,0.0,GLUCUROLACTONE
16,HYDROXYPROGESTERONE,InChI=1S/C21H30O3/c1-13(22)21(24)11-8-18-16-5-...,DBPWSSGDRRHUNT-CEGNMAFCSA-N,CHEMBL1062,0,138037,138037.0,InChI=1S/C21H30O3/c1-13(22)21(24)11-8-18-16-5-...,DBPWSSGDRRHUNT-CEGNMAFCSA-N,CHEMBL1062,0.0,HYDROXYPROGESTERONE
22,METHYL ALCOHOL,"InChI=1S/CH4O/c1-2/h2H,1H3",OKKJLVBELUTLKV-UHFFFAOYSA-N,CHEMBL14688,0,13617,13617.0,"InChI=1S/CH4O/c1-2/h2H,1H3",OKKJLVBELUTLKV-UHFFFAOYSA-N,CHEMBL14688,0.0,METHYL ALCOHOL
24,MONOETHANOLAMINE,"InChI=1S/C2H7NO/c3-1-2-4/h4H,1-3H2",HZAXFHJVJLSVMW-UHFFFAOYSA-N,CHEMBL104943,0,172066,172066.0,"InChI=1S/C2H7NO/c3-1-2-4/h4H,1-3H2",HZAXFHJVJLSVMW-UHFFFAOYSA-N,CHEMBL104943,0.0,MONOETHANOLAMINE
25,OMEGA-3-ACID ETHYL ESTERS,InChI=1S/C24H36O2.C22H34O2/c1-3-5-6-7-8-9-10-1...,DTMGIJFHGGCSLO-FIAQIACWSA-N,CHEMBL1200711,4,674662,674662.0,InChI=1S/C24H36O2.C22H34O2/c1-3-5-6-7-8-9-10-1...,DTMGIJFHGGCSLO-FIAQIACWSA-N,CHEMBL1200711,4.0,OMEGA-3-ACID ETHYL ESTERS
30,PYRITHIONE ZINC,InChI=1S/2C5H4NOS.Zn/c2*7-6-4-2-1-3-5(6)8;/h2*...,PICXIOQBANWBIZ-UHFFFAOYSA-N,CHEMBL3392049,4,1826000,1826000.0,InChI=1S/2C5H4NOS.Zn/c2*7-6-4-2-1-3-5(6)8;/h2*...,PICXIOQBANWBIZ-UHFFFAOYSA-N,CHEMBL3392049,4.0,PYRITHIONE ZINC
33,SILIBININ,InChI=1S/C25H22O10/c1-32-17-6-11(2-4-14(17)28)...,SEBFKMXJBCUCAI-UHFFFAOYSA-N,CHEMBL1401508,2,825392,825392.0,InChI=1S/C25H22O10/c1-32-17-6-11(2-4-14(17)28)...,SEBFKMXJBCUCAI-UHFFFAOYSA-N,CHEMBL1401508,2.0,SILIBININ
35,TEPRENONE,InChI=1S/2C23H38O/c2*1-19(2)11-7-12-20(3)13-8-...,DJAHKBBSJCDSOZ-AJLBTXRUSA-N,CHEMBL3989432,3,2197303,2197303.0,InChI=1S/2C23H38O/c2*1-19(2)11-7-12-20(3)13-8-...,DJAHKBBSJCDSOZ-AJLBTXRUSA-N,CHEMBL3989432,3.0,TEPRENONE


In [120]:
# Update dictionary, for those with parents
for row in spl_names_pref_name_result_df.loc[~spl_names_pref_name_result_df['parent_molregno'].isnull(),:].iterrows():
    for key in mapping_dict.keys():
        try:
            mapping_dict[key]['parent_inchi_key']
            continue
        except KeyError:
            rxnorm_concept = mapping_dict[key]['rxnorm_concept']
            try: 
                if rxnorm_concepts[rxnorm_concept]['MTHSPL'].upper() == row[1]['pref_name']:
                    mapping_dict[key]['parent_molregno']= row[1]['parent_molregno']
                    mapping_dict[key]['parent_inchi']= row[1]['parent_inchi']
                    mapping_dict[key]['parent_inchi_key']= row[1]['parent_inchi_key']
                    mapping_dict[key]['parent_chembl_id']= row[1]['parent_chembl_id']
                    mapping_dict[key]['parent_max_phase']= row[1]['parent_max_phase']
                    mapping_dict[key]['mapping_notes'] = 'mapped on rxnorm MTHSPL synonym to chembl pref_name, chose compounds with highest phase'
            
            except KeyError:
                continue


In [121]:
# Update dictionary, for those without parents
for row in spl_names_pref_name_result_df.loc[spl_names_pref_name_result_df['parent_molregno'].isnull(),:].iterrows():
    for key in mapping_dict.keys():
        try:
            mapping_dict[key]['parent_inchi_key']
            continue
        except KeyError:
            rxnorm_concept = mapping_dict[key]['rxnorm_concept']
            try: 
                if rxnorm_concepts[rxnorm_concept]['MTHSPL'].upper() == row[1]['pref_name']:
                    mapping_dict[key]['parent_molregno']= row[1]['molregno']
                    mapping_dict[key]['parent_inchi']= row[1]['standard_inchi']
                    mapping_dict[key]['parent_inchi_key']= row[1]['standard_inchi_key']
                    mapping_dict[key]['parent_chembl_id']= row[1]['chembl_id']
                    mapping_dict[key]['parent_max_phase']= row[1]['max_phase']
                    mapping_dict[key]['mapping_notes'] = 'mapped on rxnorm MTHSPL synonym to chembl pref_name, chose compounds with highest phase'
            
            except KeyError:
                continue

In [122]:
# Handy bit check mapped
still_unassigned = []
for key in mapping_dict.keys():
    try:
        mapping_dict[key]['parent_chembl_id']
    except KeyError:
        still_unassigned.append(mapping_dict[key]['rxnorm_concept'])
print(len(still_unassigned))

716


In [123]:
###### 6. RxNorm synonyms (excluding NDFRT, SNOMED_US) against ChEMBL synonyms, max_phase = 4

In [124]:
# Make string of RxNorm drug names currently unmapped
still_unassigned_ids = []
for key in mapping_dict.keys():
    try:
        mapping_dict[key]['parent_inchi_key']
    except KeyError:
        still_unassigned_ids.append(key)
joined_names_query6 = ', '.join(['"'+j+'"' for j in [mapping_dict[key]['rxnorm_name'].upper() for key in still_unassigned_ids] if (not '(' in j and not ')' in j)])    

In [125]:
len(still_unassigned_ids)

716

In [126]:
level6_currently_unmapped_rxnorm_ids = [mapping_dict[key]['rxnorm_concept'] for key in still_unassigned_ids]
len(level6_currently_unmapped_rxnorm_ids)

716

In [127]:
# Other synonyms
# This makes a list of all the synonyms for using in the query later
# Also makes a dictionary of synonym to rxnorm concept id, to check where same name is related to multiple concepts
all_synonyms = []
all_synonyms_dict = {}
for concept in rxnorm_concepts.keys():
    if concept in level6_currently_unmapped_rxnorm_ids:
        names = [rxnorm_concepts[concept][key] for key in rxnorm_concepts[concept].keys() if (key != 'NDFRT' and key != 'SNOMED_US' and key != 'CVX')]
        subset_names = [name for name in names if (not '(' in name and not ')' in name and not "'" in name)]
        all_synonyms.append(subset_names)
        for synonym in subset_names:
            try:
                all_synonyms_dict[synonym.upper()].add(concept)
            except KeyError:
                all_synonyms_dict[synonym.upper()] = {concept}


In [128]:
# These are the rxnorm concepts that have an identical synonym
# # Doesn't matter until there are multiple mappings
[all_synonyms_dict[key] for key in all_synonyms_dict.keys() if len(all_synonyms_dict[key])>1]

[{560, 743}, {7094, 1426865}, {9511, 350141}]

In [129]:
all_synonyms_formatted = []
for name in [i for i in itertools.chain(*all_synonyms)]:
    try:
        name.encode('latin-1')
        all_synonyms_formatted.append(name.upper())
    except UnicodeError:
        continue

In [130]:
len(set(all_synonyms_formatted))

1282

In [131]:
all_synonyms_query = """select distinct 
md.chembl_id
, md.max_phase
, md.pref_name
, syn.synonyms
, str.standard_inchi
, str.standard_inchi_key
, md.molregno
, hier.parent_molregno
, md2.pref_name as parent_pref_name
, md2.chembl_id as parent_chembl_id
, str2.standard_inchi as parent_inchi
, str2.standard_inchi_key as parent_inchi_key
, md2.max_phase as parent_max_phase
from molecule_dictionary md
join molecule_synonyms syn on md.molregno = syn.molregno
left join compound_structures str on str.molregno = md.molregno
left join molecule_hierarchy hier on hier.molregno = md.molregno
left join compound_structures str2 on str2.molregno = hier.parent_molregno
left join molecule_dictionary md2 on md2.molregno = hier.parent_molregno
where syn.synonyms in ({0})
and syn.syn_type != 'TRADE_NAME'
and md.max_phase=4
""".format(', '.join(['"'+str(j)+'"' for j in set(all_synonyms_formatted)]))

In [132]:
all_synonyms_result = do_query(all_synonyms_query)

In [133]:
len(all_synonyms_result)

24

In [134]:
all_synonyms_result_df = pd.DataFrame(list(all_synonyms_result), columns = ['chembl_id','max_phase', 'pref_name','synonyms', 'standard_inchi', 'standard_inchi_key', 'molregno', 'parent_molregno', 'parent_pref_name', 'parent_chembl_id', 'parent_inchi', 'parent_inchi_key', 'parent_max_phase'])

In [135]:
all_synonyms_result_df.loc[all_synonyms_result_df['parent_molregno'].isnull(),:]

,chembl_id,max_phase,pref_name,synonyms,standard_inchi,standard_inchi_key,molregno,parent_molregno,parent_pref_name,parent_chembl_id,parent_inchi,parent_inchi_key,parent_max_phase


In [136]:
all_synonyms_result_df

,chembl_id,max_phase,pref_name,synonyms,standard_inchi,standard_inchi_key,molregno,parent_molregno,parent_pref_name,parent_chembl_id,parent_inchi,parent_inchi_key,parent_max_phase
0,CHEMBL373742,4,VASOPRESSIN,Vasopressin,InChI=1S/C46H65N15O12S2/c47-27-22-74-75-23-33(...,KBZOIRJILGZLEJ-LGYYRGKSSA-N,366860,366860,VASOPRESSIN,CHEMBL373742,InChI=1S/C46H65N15O12S2/c47-27-22-74-75-23-33(...,KBZOIRJILGZLEJ-LGYYRGKSSA-N,4
1,CHEMBL457299,4,MACROGOL,Macrogol,"InChI=1S/C2H6O2/c3-1-2-4/h3-4H,1-2H2",LYCAIKOWRPUZTN-UHFFFAOYSA-N,466350,466350,MACROGOL,CHEMBL457299,"InChI=1S/C2H6O2/c3-1-2-4/h3-4H,1-2H2",LYCAIKOWRPUZTN-UHFFFAOYSA-N,4
2,CHEMBL1200542,4,DESOXYCORTICOSTERONE ACETATE,Desoxycortone,InChI=1S/C23H32O4/c1-14(24)27-13-21(26)20-7-6-...,VPGRYOFKCNULNK-ACXQXYJUSA-N,674493,674493,DESOXYCORTICOSTERONE ACETATE,CHEMBL1200542,InChI=1S/C23H32O4/c1-14(24)27-13-21(26)20-7-6-...,VPGRYOFKCNULNK-ACXQXYJUSA-N,4
3,CHEMBL1200574,4,SODIUM CHLORIDE,"Sodium chloride, hypertonic",InChI=1S/ClH.Na/h1H;/q;+1/p-1,FAPWRFPIFSIZLT-UHFFFAOYSA-M,674525,674525,SODIUM CHLORIDE,CHEMBL1200574,InChI=1S/ClH.Na/h1H;/q;+1/p-1,FAPWRFPIFSIZLT-UHFFFAOYSA-M,4
4,CHEMBL1200661,4,UNOPROSTONE ISOPROPYL,Unoprostone Isopropyl,InChI=1S/C25H44O5/c1-4-5-6-7-10-13-20(26)16-17...,XXUPXHKCPIKWLR-JHUOEJJVSA-N,674612,674612,UNOPROSTONE ISOPROPYL,CHEMBL1200661,InChI=1S/C25H44O5/c1-4-5-6-7-10-13-20(26)16-17...,XXUPXHKCPIKWLR-JHUOEJJVSA-N,4
5,CHEMBL1201648,4,AMYLASE (PANCRELIPASE),Amylase,None,None,675380,675380,AMYLASE (PANCRELIPASE),CHEMBL1201648,None,None,4
6,CHEMBL1201600,4,RABIES IMMUNE GLOBULIN,Rabies immunoglobulin,None,None,675453,675453,RABIES IMMUNE GLOBULIN,CHEMBL1201600,None,None,4
7,CHEMBL1201600,4,RABIES IMMUNE GLOBULIN,Rabies Immune Globulin,None,None,675453,675453,RABIES IMMUNE GLOBULIN,CHEMBL1201600,None,None,4
8,CHEMBL1201471,4,HYDROXYPROPYL CELLULOSE,Hydroxypropyl Cellulose,None,None,675538,675538,HYDROXYPROPYL CELLULOSE,CHEMBL1201471,None,None,4
9,CHEMBL1201651,4,PROTAMINE SULFATE,Protamine,None,None,675563,675563,PROTAMINE SULFATE,CHEMBL1201651,None,None,4


In [137]:
len(all_synonyms_result_df['parent_molregno'].drop_duplicates())

21

In [141]:
# Make a dictionary of each rxnorm id with the molregnos retrieved base on synonym mapping, to see if multiple molregnos per rxnorm are retrieved
rxnorm_molregno_dict = {}
for item in all_synonyms_result:
    
    parent_molregno = item[7]
    synonym = item[3].upper()
    current_rxnorm_ids = all_synonyms_dict[synonym]
    for rxnorm_id in current_rxnorm_ids:
        try:
            rxnorm_molregno_dict[rxnorm_id].add(parent_molregno)
        except KeyError:
            rxnorm_molregno_dict[rxnorm_id] = {parent_molregno}

In [142]:
# There are rxnorm concepts mapped to more than one molregno, need manual inspection
[(key,rxnorm_molregno_dict[key]) for key in rxnorm_molregno_dict.keys() if len(rxnorm_molregno_dict[key])>1]

[]

In [143]:
# Need to check if I get more than 2 molregno per rxnorm_id, these are parent_molregnos
multiples = list(itertools.chain(*[rxnorm_molregno_dict[key] for key in rxnorm_molregno_dict.keys() if len(rxnorm_molregno_dict[key])>1]))
multiples

[]

In [144]:
# Manual inspection, I'm unsure about Pristinamycin and Follicle Stimulating Hormone, seem to be multiple component concepts

In [146]:
# update dictionary
for row in all_synonyms_result_df.iterrows():
    parent_molregno = row[1]['parent_molregno']
    synonym = row[1]['synonyms']
    if synonym in ['Follicle Stimulating Hormone', 'Pristinamycin']:
        print('skipped: '+ synonym)
        continue
    
    for key in mapping_dict.keys():
        try:
            mapping_dict[key]['parent_inchi_key']
            continue
        except KeyError:
            rxnorm_concept = mapping_dict[key]['rxnorm_concept']
            # rxnorm to synonyms dict
            names = [rxnorm_concepts[rxnorm_concept][syn].upper() for syn in rxnorm_concepts[rxnorm_concept].keys() if (syn != 'NDFRT' and syn != 'SNOMED_US' and syn != 'CVX')]
            if synonym.upper() in names:
                mapping_dict[key]['parent_molregno']= row[1]['parent_molregno']
                mapping_dict[key]['parent_inchi']= row[1]['parent_inchi']
                mapping_dict[key]['parent_inchi_key']= row[1]['parent_inchi_key']
                mapping_dict[key]['parent_chembl_id']= row[1]['parent_chembl_id']
                mapping_dict[key]['parent_max_phase']= row[1]['parent_max_phase']
                mapping_dict[key]['mapping_notes'] = 'mapped rxnorm synonym to chembl synonym, max_phase=4'


skipped: Follicle Stimulating Hormone
skipped: Pristinamycin


In [147]:
# Handy bit check mapped
still_unassigned = []
for key in mapping_dict.keys():
    try:
        mapping_dict[key]['parent_inchi_key']
    except KeyError:
        still_unassigned.append(mapping_dict[key]['rxnorm_concept'])
print(len(still_unassigned))

697


In [148]:
# Number of compounds mapped
assigned = set()
for key in mapping_dict.keys():
    try:
        mapping_dict[key]['parent_inchi_key']
        mapping_dict[key]['parent_chembl_id']
        assigned.add(key)
    except KeyError:
        continue
print(len(assigned))

832


In [149]:
### close db connections
conn.close()
calculon_chembl.close()
server.close()

In [150]:
######## Need to update mapped concepts db

In [167]:
conn = sqlite.connect(mapped_compounds_db)
cur = conn.cursor()

In [152]:
cur.execute('pragma table_info(compound_structures)').fetchall()

[(0, 'aeolus_concept', 'integer', 0, None, 0),
 (1, 'rxnorm_name', 'text', 0, None, 0),
 (2, 'rxnorm_concept', 'integer', 0, None, 0),
 (3, 'drugbank_id', 'text', 0, None, 0),
 (4, 'original_drugbank_unichem_inchi', 'text', 0, None, 0),
 (5, 'original_drugbank_unichem_inchi_key', 'text', 0, None, 0),
 (6, 'mapped_parent_chembl_id', 'text', 0, None, 0),
 (7, 'mapped_parent_standard_inchi', 'text', 0, None, 0),
 (8, 'mapped_parent_standard_inchi_key', 'text', 0, None, 0),
 (9, 'mapped_parent_molregno', 'integer', 0, None, 0),
 (10, 'mapped_parent_max_phase', 'integer', 0, None, 0),
 (11, 'mapping_notes', 'text', 0, None, 0),
 (12, 'notes', 'text', 0, None, 0)]

In [169]:
cur.execute('select count(distinct rxnorm_concept) from compound_structures').fetchall()

[(2658,)]

In [168]:
cur.execute('select count(distinct rxnorm_concept) from compound_structures where mapped_parent_chembl_id is null').fetchall()

[(0,)]

In [155]:
insert_row = """insert into compound_structures 
(aeolus_concept
, rxnorm_name
, rxnorm_concept
, drugbank_id
, mapped_parent_chembl_id
, mapped_parent_standard_inchi
, mapped_parent_standard_inchi_key
, mapped_parent_molregno 
, mapped_parent_max_phase
, mapping_notes)
values 
(?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"""

for key in mapping_dict.keys():
    try:
        mapping_dict[key]['parent_chembl_id']
        cur.execute(insert_row, (key
            , mapping_dict[key]['rxnorm_name']
            , mapping_dict[key]['rxnorm_concept']
            , mapping_dict[key]['drugbank_id']
            , mapping_dict[key]['parent_chembl_id']
            , mapping_dict[key]['parent_inchi']
            , mapping_dict[key]['parent_inchi_key']
            , mapping_dict[key]['parent_molregno']
            , mapping_dict[key]['parent_max_phase']
            , mapping_dict[key]['mapping_notes']))
    except KeyError:
        continue

In [158]:
# Number of compounds mapped to a ChEMBL ID
cur.execute('select count(*) from compound_structures').fetchall()

[(2669,)]

In [157]:
cur.execute('select * from compound_structures where mapped_parent_chembl_id is null').fetchall()

[]

In [172]:
# Number of concepts with a structure
cur.execute('select count(distinct rxnorm_concept) from compound_structures where mapped_parent_standard_inchi_key is not null').fetchall()

[(2282,)]

In [171]:
# Number of compounds without drugbank ID that I mapped to ChEMBL
cur.execute('select count(distinct rxnorm_concept) from compound_structures where drugbank_id is null and mapped_parent_chembl_id is not null').fetchall()

[(497,)]

In [177]:
# Number of approved drugs
cur.execute('select count(distinct rxnorm_concept) from compound_structures where mapped_parent_max_phase=4').fetchall()

[(1965,)]

In [178]:
cur.execute('select count(distinct rxnorm_concept) from compound_structures where mapped_parent_max_phase=4').fetchall()

[(2014,)]

In [164]:
conn.commit()

In [165]:
conn.close()

In [84]:
### close db connections
conn.close()
calculon_chembl.close()
server.close()

NameError: name 'calculon_chembl' is not defined